In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

In [3]:
df = pd.read_excel('250110_BMED_train data_v1.xlsx',sheet_name='Sheet2')
df[df['exp']==20]

,exp,T,V,E,Ci,Ki,t,SEC,JLA,CF_LA,...,CF_K,CA_K,CB_K,VF,VA,VB,NF_LA,NA_LA,NF_K,NB_K
120,20,26.3,30,0.25,3.0,6.0,0.0,NaN,NaN,2.893057,...,5.268312,0.009215,0.039988,1.00,1.00,6.00,3.000000,0.000000,6.000000,0.000000
121,20,26.0,30,0.25,3.0,6.0,1.0,1.913116,122.123449,2.738909,...,4.136552,0.013755,0.119186,1.08,0.99,5.85,3.067367,-0.067367,5.087940,0.912060
122,20,26.8,30,0.25,3.0,6.0,3.0,1.348034,321.875207,2.372142,...,2.530328,0.020611,0.357907,1.08,1.03,5.80,2.656615,0.343385,3.112292,2.887708
123,20,27.4,30,0.25,3.0,6.0,4.0,1.219475,466.897375,2.335240,...,1.922648,0.022295,0.483722,0.98,1.06,5.80,2.373132,0.626868,2.145881,3.854119
124,20,27.6,30,0.25,3.0,6.0,5.0,1.070469,614.903438,2.038700,...,1.397192,0.023384,0.607582,0.88,1.15,5.85,1.860374,1.139626,1.400292,4.599708
125,20,27.0,30,0.25,3.0,6.0,6.0,0.889761,886.800104,1.701408,...,0.940736,0.024038,0.719509,0.73,1.27,5.80,1.287940,1.712060,0.782115,5.217885
126,20,26.4,30,0.25,3.0,6.0,7.0,0.785846,874.256920,1.218024,...,0.633594,0.024474,0.812147,0.57,1.39,5.90,0.719938,2.280062,0.411307,5.588693
127,20,25.7,30,0.25,3.0,6.0,8.0,0.614033,1572.360316,0.583133,...,0.264175,0.024638,0.882614,0.38,1.50,6.00,0.229782,2.770218,0.114329,5.885671


In [4]:
t = []
T = []
NF_LA = []
NF_K = []
VF = []
VA = []
VB = []

for i in range(34):
    fla = interp1d(df[df['exp']==i+1]['t'],df[df['exp']==i+1]['NF_LA'], kind='linear')
    fk = interp1d(df[df['exp']==i+1]['t'],df[df['exp']==i+1]['NF_K'], kind='linear')
    fT = interp1d(df[df['exp']==i+1]['t'],df[df['exp']==i+1]['T'], kind='linear')
    fVF = interp1d(df[df['exp']==i+1]['t'],df[df['exp']==i+1]['VF'], kind='linear')
    fVA = interp1d(df[df['exp']==i+1]['t'],df[df['exp']==i+1]['VA'], kind='linear')
    fVB = interp1d(df[df['exp']==i+1]['t'],df[df['exp']==i+1]['VB'], kind='linear')
    tn = np.arange(0,df[df['exp']==i+1]['t'].iloc[-1],0.1)
    lan = fla(tn)
    kn = fk(tn)
    Tn = fT(tn)
    vfn = fVF(tn)
    van = fVA(tn)
    vbn = fVB(tn)
    t.append(tn)
    NF_LA.append(lan)
    NF_K.append(kn)
    T.append(Tn)
    VF.append(vfn)
    VA.append(van)
    VB.append(vbn)

In [5]:
NA_LA = []
NB_K = []

for i in range(len(NF_LA)):
    nla = NF_LA[i][0] - NF_LA[i]
    nk = NF_K[i][0] - NF_K[i]
    NA_LA.append(nla)
    NB_K.append(nk)

In [6]:
dNLA = []
dNK = []
dVF = []
dVA = []
dVB = []

for i in range(len(NF_LA)):
    dnla = np.zeros(len(NF_LA[i]))
    dnk = np.zeros(len(NF_LA[i]))
    dvf = np.zeros(len(NF_LA[i]))
    dva = np.zeros(len(NF_LA[i]))
    dvb = np.zeros(len(NF_LA[i]))
    for j in range(len(NF_LA[i])):
        if j == 0:
            dnla[j] = -(-3*NF_LA[i][j]+4*NF_LA[i][j+1]-NF_LA[i][j+2])/2/(t[i][j+1] - t[i][j])
            dnk[j] = -(-3*NF_K[i][j]+4*NF_K[i][j+1]-NF_K[i][j+2])/2/(t[i][j+1] - t[i][j])
            dvf[j] = -(-3*VF[i][j]+4*VF[i][j+1]-VF[i][j+2])/2/(t[i][j+1] - t[i][j])
            dva[j] = -(-3*VA[i][j]+4*VA[i][j+1]-VA[i][j+2])/2/(t[i][j+1] - t[i][j])
            dvb[j] = -(-3*VB[i][j]+4*VB[i][j+1]-VB[i][j+2])/2/(t[i][j+1] - t[i][j])
        elif j == len(NF_LA[i])-1:
            dnla[j] = -(-3*NF_LA[i][j]+4*NF_LA[i][j-1]-NF_LA[i][j-2])/2/(t[i][j] - t[i][j-1])
            dnk[j] = -(-3*NF_K[i][j]+4*NF_K[i][j-1]-NF_K[i][j-2])/2/(t[i][j] - t[i][j-1])
            dvf[j] = -(-3*VF[i][j]+4*VF[i][j-1]-VF[i][j-2])/2/(t[i][j] - t[i][j-1])
            dva[j] = -(-3*VA[i][j]+4*VA[i][j-1]-VA[i][j-2])/2/(t[i][j] - t[i][j-1])
            dvb[j] = -(-3*VB[i][j]+4*VB[i][j-1]-VB[i][j-2])/2/(t[i][j] - t[i][j-1])
        else:
            dnla[j] = (NF_LA[i][j+1] - NF_LA[i][j-1])/(t[i][j+1] - t[i][j-1])
            dnk[j] = (NF_K[i][j+1] - NF_K[i][j-1])/(t[i][j+1] - t[i][j-1])
            dvf[j] = (VF[i][j+1] - VF[i][j-1])/(t[i][j+1] - t[i][j-1])
            dva[j] = (VA[i][j+1] - VA[i][j-1])/(t[i][j+1] - t[i][j-1])
            dvb[j] = (VB[i][j+1] - VB[i][j-1])/(t[i][j+1] - t[i][j-1])
    dNLA.append(dnla)
    dNK.append(dnk)
    dVF.append(dvf)
    dVA.append(dva)
    dVB.append(dvb)

In [7]:
dNLA

[array([ 0.00834347, -0.00834347, -0.00834347, -0.00834347, -0.00834347,
        -0.00834347, -0.00834347, -0.00834347, -0.00834347, -0.00834347,
        -0.01831778, -0.02829209, -0.02829209, -0.02829209, -0.02829209,
        -0.02829209, -0.02829209, -0.02829209, -0.02829209, -0.02829209,
        -0.04793796, -0.06758382, -0.06758382, -0.06758382, -0.06758382,
        -0.06758382, -0.06758382, -0.06758382, -0.06758382, -0.06758382,
        -0.09314123, -0.11869865, -0.11869865, -0.11869865, -0.11869865,
        -0.11869865, -0.11869865, -0.11869865, -0.11869865, -0.11869865,
        -0.13903267, -0.15936669, -0.15936669, -0.15936669, -0.15936669,
        -0.15936669, -0.15936669, -0.15936669, -0.15936669, -0.15936669,
        -0.15936669]),
 array([-0.00313688,  0.00313688,  0.00313688,  0.00313688,  0.00313688,
         0.00313688,  0.00313688,  0.00313688,  0.00313688,  0.00313688,
         0.00601605,  0.00889523,  0.00889523,  0.00889523,  0.00889523,
         0.00889523,  0.0088